# Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
# import matplotlib.pyplot as plt
# plt.style.use('seaborn')

In [34]:
# Import file
data = pd.read_csv('medical_examination.csv', index_col='id')
data.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


## Task 1

Create a chart similar to examples/Figure_1.png, where we show the counts of good and bad outcomes for the cholesterol, gluc, alco, active, and smoke variables for patients with cardio=1 and cardio=0 in different panels.

In [50]:
def calc_overweight(df):
    '''
    A simple function to caclulate BMI
    '''
    df['overweight'] = (df['weight'] / ((df['height']/100) ** 2)).to_numpy()
    df['overweight'] = np.where(df['overweight'] < 25, 0, 1)

    # Normalize the data by making 0 always good and 1 always bad. If the value of cholesterol         or gluc is 1, make the value 0. If the value is more than 1, make the value 1.
    df['cholesterol'] = np.where(df['cholesterol'] <= 1, 0, 1)
    df['gluc'] = np.where(df['gluc'] <= 1, 0, 1)
    return df


In [35]:
data1 = data.copy()

In [49]:
test = calc_overweight(data1)
test.head(10)

TypeError: where() got an unexpected keyword argument 'inplace'

In [42]:
test2 = test.iloc[:,7:].reset_index()
test2.head()

,id,gluc,smoke,alco,active,cardio,overweight
0,0,0,0,0,1,0,0
1,1,0,0,0,1,1,1
2,2,0,0,0,0,1,0
3,3,0,0,0,1,1,1
4,4,0,0,0,0,0,0


In [46]:
long_df = pd.wide_to_long(test2, stubnames='ht', i=['id','alco'], j='cardi').reset_index()
long_df.head()

,id,alco,cardi,smoke,cardio,active,overweight,gluc,ht


## Create dataframe for cardio_0 and 1

In [5]:
cardio_0 = data[data['cardio'] == 0].iloc[:,-7:].set_index('cardio')
cardio_0.head()

,cholesterol,gluc,smoke,alco,active,overweight
cardio,,,,,,
0,0,0,0,0,1,0
0,0,0,0,0,0,0
0,1,1,0,0,0,1
0,1,0,0,0,1,1
0,0,0,0,0,1,1


In [8]:
cardio_1 = data[data['cardio'] == 1].iloc[:,-7:]
cardio_1.head()

,cholesterol,gluc,smoke,alco,active,cardio,overweight
id,,,,,,,
1,1,0,0,0,1,1,1
2,1,0,0,0,0,1,0
3,0,0,0,0,1,1,1
12,1,1,0,0,1,1,1
24,0,0,0,0,0,1,1


In [7]:
# put the needed columns to a list
plt_columns = cardio_1.iloc[:,-7:].columns.sort_values().to_list()
plt_columns

['active', 'alco', 'cholesterol', 'gluc', 'overweight', 'smoke']

In [12]:
sns.catplot(x=['cardio','cholesterol'], data=test, kind='count')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().